In [1]:
# Not needed if pysleep is pip installed (i.e. you probably dont need these lines)
import sys, os
file_dir = os.path.abspath('')
sys.path.insert(0, file_dir+'/../')

In [7]:
#%% Import the tools we need
from mednickdb_pysleep import process_sleep_record, scorefiles, sleep_architecture
import pandas as pd
import numpy as np
import yaml
import mne
from scipy.io import savemat
from wonambi import Dataset
from wonambi.trans import math, timefrequency
import warnings
import datetime
warnings.filterwarnings("ignore")

In [12]:
#setup location of files and save locations
edf_base_path = 'C:/Users/bdyet/Desktop/ExampleStudy2/raw_sleep_eeg/'
scorefile_base_path = 'C:/Users/bdyet/Desktop/ExampleStudy2/sleep_scoring/'
edf_filenames = ['Sid1_edf.edf']
scorefile_filenames = ['sid1.csv']
ids = ['1'] #these are the ids for each subject, they conrespond to each edf/scorefile above
study_settings_path = 'C:/Users/bdyet/Desktop/ExampleStudy2/choc_type_study_settings.yaml'
save_base_path = 'C:/Users/bdyet/Desktop/ExampleStudy2/'
max_freq = 35
epoch_len = 3 #avergae power evey 3 seconds

In [60]:
#%% loop through edf's, extract band power, spindles and slow osc (per stage)
edf_filepaths = [edf_base_path + edf_filename for edf_filename in edf_filenames]
scoring_filepaths = [scorefile_base_path + scorefile_filename for scorefile_filename in scorefile_filenames]
features_averages_cont = []
band_power_averages_cont = []
features_df_cont = []

study_settings = yaml.safe_load(open(study_settings_path,'r+'))

for edf_filepath, scorefile_filepath in zip(edf_filepaths, scoring_filepaths):   
    print('Working on',edf_filepath)
    
        #%%Get stage info
    epoch_stages, epochoffset, starttime = scorefiles.extract_epochstages_from_scorefile(scorefile_filepath, 
                                                                                         study_settings['stage_map'])
    epoch_stages = scorefiles.score_wake_as_waso_wbso_wase(epoch_stages)

    #get the start and end of where we want to extract spindles from (lights off->lights on)
    lights_off_secs, end_time, \
    _, epoch_stages = sleep_architecture.lights_on_off_and_sleep_latency(epoch_stages,
                                                                         epoch_sync_offset_seconds=epochoffset)
    
    edf = mne.io.read_raw_edf(edf_filepath)
    chans_to_consider = [v for v in study_settings['known_eeg_chans'].keys() if v in edf.ch_names]
    
    d = Dataset(edf_filepath)
    if not (lights_off_secs is None or lights_off_secs >= 0):
        raise error_handling.EEGError('Epochoffset is negative!'+str(lights_off_secs))
    if not ((end_time is None) or (end_time <= d.header['n_samples']/d.header['s_freq'])):
        raise error_handling.EEGError("end time ("+ str(end_time) +") larger than record end!"+str(d.header['n_samples']/d.header['s_freq']))
    data = d.read_data(begtime=lights_off_secs, endtime=end_time, chan=chans_to_consider)
    power = timefrequency(data, method='spectrogram')
    abs_power = math(power, operator_name='abs')
    chan_time_freq = abs_power.data[0]
    all_chans = np.ones((chan_time_freq.shape[0],), dtype=bool)
    time_axis = np.round(abs_power.axis['time'][0], 2)
    freq_axis = np.round(abs_power.axis['freq'][0], 2)
    chan_axis = abs_power.axis['chan'][0]
    freq_binsize = freq_axis[1] - freq_axis[0]
    assert epoch_len > 0, "epoch len must be greater than zero"
    times = np.arange(0,time_axis[-1],epoch_len)
    cont = []
    freq_mask = freq_axis < max_freq
    time_slice_cont = []
    for win_start in times:
        time_mask = (win_start <= time_axis) & (time_axis < win_start + epoch_len)
        idx = np.ix_(all_chans, time_mask, all_freqs)
        chan_epoch_per_band = chan_time_freq[idx].mean(axis=1) / freq_binsize
        time_slice_cont.append(chan_epoch_per_band)
    all_data = np.stack(time_slice_cont)
    all_data = all_data.swapaxes(1,2)
    savemat(save_base_path+os.path.split(edf_filepath)[1][:-4]+'_power.mat', mdict={'power': all_data, 'time_axis':times, 'freq_axis': freq_axis[freq_mask], 'lights_off_secs':lights_off_secs})
    print('\tDone, .mat file saved')
    

    
    

Working on C:/Users/bdyet/Desktop/ExampleStudy2/raw_sleep_eeg/Sid1_edf.edf
	Done, .mat file saved
